In [49]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
from src.constants import headers

In [51]:
import requests
from bs4 import BeautifulSoup

In [52]:
response = requests.get('https://pk.mpei.ru/inform/list.html', headers=headers)

In [87]:
response

<Response [200]>

In [88]:
soup = BeautifulSoup(response.text)

In [89]:
all_group_links = soup.find_all('div', {'class':'groupFilterFormO'})[0].find_all('table')[0].find_all('a', {'class':"competitive-group"})
len(all_group_links)

369

In [45]:
# all_group_links = soup.find_all('tbody', {'class':'groupFilterMoscow'})[0].find_all('a', {'class':"competitive-group"})
# len(all_group_links)

271

In [90]:
target_links = [l for l in all_group_links if 'bacc' in l.get('href')]
len(target_links)

36

In [4]:
from mpei.scrape_data import get_all_target_links
target_links = get_all_target_links()

In [25]:
from src.classes import ListUniversityDirection,UniversityDirection, StudentsPrepWorkflow
students = []
lud = ListUniversityDirection()
sw = StudentsPrepWorkflow()

In [6]:
target_links[0]

<a class="competitive-group listFilterBudget" href="/inform/list581bacc.html" target="list"><img border="0" height="8" src="/img/orange.gif" width="4"/> конкурсный список поступающих на основные места в рамках КЦП (бюджет, зачислениe 4-9 августа 2023г.)<br/></a>

In [7]:
link = target_links[0]
url = link.get('href')

In [8]:
response = requests.get('https://pk.mpei.ru/'+url, headers=headers)

In [9]:
soup = BeautifulSoup(response.text)

In [10]:
dir = soup.find_all('div',{'class':"competitive-group"})[0].text
dir

'Прикладная математика и информатика МПОВМКС ИВТИ бюджет (Очная) '

In [11]:
places_text = [x for x in soup.find_all('div',{'class':"title1"})[0].getText("\n").split('\n') if 'Количество вакантных мест' in x][0]
places_count = int(places_text.replace("Количество вакантных мест: ",""))
print(places_text, places_count)

Количество вакантных мест: 50 50


In [12]:
#/inform/list581bacc.html
key = url.replace('/inform/list','').replace('.html','')
key

'581bacc'

In [13]:
import pandas as pd
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
df = pd.read_html(response.text)[0]

In [14]:
df

СНИЛС или Рег.номер Сумма Сумма без ИД     Баллы ВИ*  \
     СНИЛС или Рег.номер Сумма Сумма без ИД Мат. Инж.мат.   
0     СНИЛС: 16552783796   303          293            96   
1     СНИЛС: 16656357100   299          291           100   
2     СНИЛС: 16492380589   299          289           100   
3     СНИЛС: 16994776356   298          288            88   
4     СНИЛС: 19665702418   295          285            92   
...                  ...   ...          ...           ...   
2400  СНИЛС: 16005001185   146          146            52   
2401  СНИЛС: 18485803212   144          142            52   
2402  СНИЛС: 15918621695   144          140            40   
2403  СНИЛС: 15282522454   142          142            40   
2404  СНИЛС: 14669791625   140          140            40   

                                  ИД Преимущ. право**            Оригинал  \
     Инф. ИТ Проф. Инж.физ. Рус.  ИД        ч.9 ст.71 ч.10 ст.71 Оригинал   
0                       100   97  10              нет        нет      нет   
1                       100   91   8              нет        нет      нет   
2                        98   91  10              нет        нет      нет   
3                       100  100  10              нет        нет      нет   
4                       100   93  10              нет        нет       да   
...                     ...  ...  ..              ...        ...      ...   
2400                     54   40   0              нет        нет      нет   
2401                     48   42   2              нет        нет      нет   
2402                     48   52   4              нет        нет      нет   
2403                     48   54   0              нет        нет      нет   
2404                     45   55   0              нет        нет      нет   

     Договор Приоритет Приоритет высший Общежитие Примечание  
     Договор Приоритет Приоритет высший Общежитие Примечание  
0        нет         3              нет       б/о        NaN  
1        нет         1              нет       б/о        NaN  
2        нет         1              нет       б/о        NaN  
3        нет         2              нет       б/о        NaN  
4        нет         1               да       б/о        NaN  
...      ...       ...              ...       ...        ...  
2400     нет         5              нет       б/о        NaN  
2401     нет         1              нет       б/о        NaN  
2402     нет         3              нет       с/о        NaN  
2403     нет         4              нет       б/о        NaN  
2404     нет         3              нет       б/о        NaN  

[2405 rows x 15 columns]

In [15]:
df.columns = [x[1] for x in df.columns]

In [30]:
st_df = df[['СНИЛС или Рег.номер','Сумма','Оригинал','Приоритет']]
st_df.columns = ['snils','points', 'podl', 'priority']

In [37]:
st_df['snils'] = st_df['snils'].apply(lambda x: x.replace('СНИЛС: ',''))
st_df['dir_key'] = key
st_df['podl'] = st_df['podl'].apply(lambda x: True if x =='дa' else False)

In [38]:
st_df

,snils,points,podl,priority,dir_key
0,16552783796,303,False,3,581bacc
1,16656357100,299,False,1,581bacc
2,16492380589,299,False,1,581bacc
3,16994776356,298,False,2,581bacc
4,19665702418,295,False,1,581bacc
...,...,...,...,...,...
2400,16005001185,146,False,5,581bacc
2401,18485803212,144,False,1,581bacc
2402,15918621695,144,False,3,581bacc
2403,15282522454,142,False,4,581bacc


In [39]:
ud = UniversityDirection(key= key, places= places_count, properties={'name':dir, 'url':url.replace('/inform/list','')})

In [40]:
lud.update(key=ud)

In [41]:
students += sw.df2StudentWithPriority(st_df)

In [42]:
students

[StudentWithPriority(snils='16552783796', points=303, podl='нет', priority=3, dir_key='581bacc'),
 StudentWithPriority(snils='16656357100', points=299, podl='нет', priority=1, dir_key='581bacc'),
 StudentWithPriority(snils='16492380589', points=299, podl='нет', priority=1, dir_key='581bacc'),
 StudentWithPriority(snils='16994776356', points=298, podl='нет', priority=2, dir_key='581bacc'),
 StudentWithPriority(snils='19665702418', points=295, podl='да', priority=1, dir_key='581bacc'),
 StudentWithPriority(snils='17920718288', points=292, podl='нет', priority=3, dir_key='581bacc'),
 StudentWithPriority(snils='13671960681', points=292, podl='нет', priority=1, dir_key='581bacc'),
 StudentWithPriority(snils='17392606489', points=291, podl='нет', priority=2, dir_key='581bacc'),
 StudentWithPriority(snils='19830835913', points=291, podl='нет', priority=1, dir_key='581bacc'),
 StudentWithPriority(snils='16601057840', points=290, podl='да', priority=3, dir_key='581bacc'),
 StudentWithPriority(s